In [1]:
import json
import pandas as pd
import requests
import utils

In [2]:
api_key = utils.api_key()
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'

def dataframe_from_area_code(query):
    r = requests.get(url + 'query=' + query + '&key=' + api_key)
    x = r.json()
    y = x['results']
    df = pd.DataFrame(y)
    return df

global final_dataset

In [3]:
# south LA zip codes
zip_code = ['90001', '90002', '90003', '90007', '90008', '90011', '90016',
            '90018', '90037', '90043', '90044', '90047', '90059', '90061',
            '90062', '90266', '90267', '90291', '90292', '90293', '90401',
            '90402', '90403', '90404', '90405']
places = ['smoke shops', 'liquor stores', 'marijuana dispenseries', 'schools']

In [4]:
final_dataset = pd.DataFrame()

for i in range(len(places)):
    for j in range(len(zip_code)):
        a = pd.DataFrame(dataframe_from_area_code(places[i] + ' near ' + zip_code[j]))
        a['shops'] = places[i]
        a['zip'] = zip_code[j]
        final_dataset = pd.concat([final_dataset, a], axis=0, sort=True)

In [5]:
# clean up and create final dataframe
final_dataset.drop(['id', 'icon', 'place_id', 'opening_hours', 
                    'plus_code', 'reference', 'photos', 'price_level', 
                    'rating', 'types', 'user_ratings_total'], axis=1, inplace=True)
final_dataset.rename(columns = {'formatted_address': 'address', 'geometry': 'lat_long'}, inplace=True)
final_dataset.drop_duplicates(subset='address', keep='first', inplace=True)
final_dataset.reset_index(drop=True, inplace=True)
print(final_dataset.shape)
final_dataset.head()

(763, 5)


,address,lat_long,name,shops,zip
0,"626 E Manchester Ave, Los Angeles, CA 90001, U...","{'location': {'lat': 33.9597758, 'lng': -118.2...",Smoke Shop,smoke shops,90001
1,"5968 Santa Fe Ave, Huntington Park, CA 90255, ...","{'location': {'lat': 33.9878515, 'lng': -118.2...",The Spot Smoke Shop,smoke shops,90001
2,"1804 Firestone Blvd, Los Angeles, CA 90001, Un...","{'location': {'lat': 33.95994659999999, 'lng':...",TNT Tobacco Store,smoke shops,90001
3,"7610 San Pedro St, Los Angeles, CA 90003, Unit...","{'location': {'lat': 33.9709491, 'lng': -118.2...",Morelos Smoke Shop,smoke shops,90001
4,"7300 San Pedro St, Los Angeles, CA 90003, Unit...","{'location': {'lat': 33.973667, 'lng': -118.26...",D & D Smoke Shop,smoke shops,90001


In [6]:
count1 = pd.DataFrame(columns=places, index=zip_code)
count1[count1 != 0] = 0
for i in range(len(final_dataset['zip'])):
    for j in range(len(zip_code)):
        if final_dataset['zip'][i] == zip_code[j]:
            b = zip_code[j]
            for z in range(len(places)):
                if final_dataset['shops'][i] == places[z]:
                    a = count1.loc[b][places[z]]
                    count1.loc[b][places[z]] = a+1
count1                                      

,smoke shops,liquor stores,marijuana dispenseries,schools
90001,20,18,1,19
90002,7,12,0,16
90003,5,13,0,19
90007,19,20,16,18
90008,16,19,0,17
90011,2,14,0,20
90016,8,13,0,12
90018,3,6,0,13
90037,7,12,0,16
90043,5,13,0,17


# South LA Counts

In [7]:
# number of shops for every area code
print(list(count1['smoke shops']))
print(list(count1['liquor stores']))
print(list(count1['marijuana dispenseries']))
print(list(count1['schools']))

[20, 7, 5, 19, 16, 2, 8, 3, 7, 5, 2, 4, 4, 8, 0, 20, 4, 20, 4, 2, 3, 1, 0, 3, 0]
[18, 12, 13, 20, 19, 14, 13, 6, 12, 13, 16, 7, 10, 3, 0, 18, 7, 20, 11, 9, 12, 4, 0, 1, 3]
[1, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[19, 16, 19, 18, 17, 20, 12, 13, 16, 17, 19, 15, 19, 10, 13, 19, 7, 17, 16, 13, 1, 0, 0, 0, 0]
